In [ ]:
#Starts from here
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression  # LogisticRegression is not used for regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
from tqdm import tqdm

def remove_low_variance_columns(df, threshold=0.005):
    # df = df.drop(['ID','SMILES','Permeability'],axis=1)
    variances = df.var()
    
    # Columns with variance below the threshold
    low_variance_columns = variances[variances < threshold].index.tolist()
    
    df_cleaned = df.drop(columns=low_variance_columns)
    
    return df_cleaned, low_variance_columns

def features(df, target_column='Permeability', threshold=0.9):
    correlation_matrix = df.corr()
    
    features_to_drop = set()
    
    for feature in correlation_matrix.columns:
        if feature == target_column:
            continue 
        target_corr = correlation_matrix[target_column][feature]
        
        for other_feature in correlation_matrix.columns:
            if other_feature == feature or other_feature == target_column:
                continue
            
            if abs(correlation_matrix[feature][other_feature]) > threshold:
                other_target_corr = correlation_matrix[target_column][other_feature]

                if abs(other_target_corr) < abs(target_corr):
                    features_to_drop.add(other_feature)
                else:
                    features_to_drop.add(feature)
    selected_features = [col for col in df.columns if col not in features_to_drop and col != target_column]
    
    return selected_features

In [4]:
# 2D and 3D descriptors dataframes
df_desc_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Train_2d_3d_all_descriptors.csv')
df_train = df_desc_train.sort_values(by='ID')
df_train =df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_desc_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
df_desc_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Test_2d_3d_all_descriptors.csv')
df_desc_test = df_desc_test.sort_values(by='ID')
df_desc_test =df_desc_test.dropna()
df_desc_test =  df_desc_test[df_desc_train.columns]

# Fingerprints
df_fp_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/All_fingerprints_train.csv')
df_train = df_fp_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_fp_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
df_fp_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/All_fingerprints_test.csv')
df_fp_test = df_fp_test.sort_values(by='ID')
df_fp_test = df_fp_test.dropna()
df_fp_test =  df_fp_test[df_fp_train.columns]


#Smiles Embeddings
df_emb_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Embeddings/Train_MoLFormer-XL-both-10pct_model_1_fine_tuned_embeddings.csv')
df_train = df_emb_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_emb_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
df_emb_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Embeddings/Test_MoLFormer-XL-both-10pct_model_1_fine_tuned_embeddings.csv')
df_emb_test = df_emb_test.sort_values(by='ID')
df_emb_test = df_emb_test.dropna()
df_emb_test =  df_emb_test[df_emb_train.columns]

#ATomic features
df_atomic_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Atomic/Train_all_atomic_desc.csv')
df_train = df_atomic_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_atomic_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
# df_atomic_train =pd.concat( [df_train['SMILES'], df_train.select_dtypes(include=['number'])], axis=1)
df_atomic_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Atomic/Test_all_atomic_desc.csv')
df_atomic_test = df_atomic_test.sort_values(by='ID')
df_atomic_test = df_atomic_test.dropna()
df_atomic_test =  df_atomic_test[df_atomic_train.columns]


print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Data Loading completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
df_fp_test = df_fp_test[df_fp_test['ID'].isin(df_desc_test['ID'])]
df_fp_train = df_fp_train[df_fp_train['ID'].isin(df_desc_train['ID'])]

df_emb_test = df_emb_test[df_emb_test['ID'].isin(df_desc_test['ID'])]
df_emb_train = df_emb_train[df_emb_train['ID'].isin(df_desc_train['ID'])]

df_atomic_test = df_atomic_test[df_atomic_test['ID'].isin(df_desc_test['ID'])]
df_atomic_train = df_atomic_train[df_atomic_train['ID'].isin(df_desc_train['ID'])]

print(df_desc_train.shape)
print(df_desc_test.shape)
print(df_fp_train.shape)
print(df_fp_test.shape)
print(df_emb_train.shape)
print(df_emb_test.shape)
print(df_atomic_train.shape)
print(df_atomic_test.shape)

def scale_features(df_train, df_test):
    scaler = StandardScaler()
    train_features = df_train.drop(columns=['ID', 'SMILES', target_column])
    test_features = df_test.drop(columns=['ID', 'SMILES', target_column])
    scaler.fit(train_features)
    train_scaled = pd.DataFrame(scaler.transform(train_features), columns=train_features.columns, index=df_train.index)
    test_scaled = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns, index=df_test.index)
    df_train_scaled = pd.concat([df_train[['ID', 'SMILES', target_column]], train_scaled], axis=1)
    df_test_scaled = pd.concat([df_test[['ID', 'SMILES', target_column]], test_scaled], axis=1)
    return df_train_scaled, df_test_scaled

target_column = 'Permeability'
df_desc_train, df_desc_test = scale_features(df_desc_train, df_desc_test)
df_fp_train, df_fp_test = scale_features(df_fp_train, df_fp_test)
df_emb_train, df_emb_test = scale_features(df_emb_train, df_emb_test)
df_atomic_train, df_atomic_test = scale_features(df_atomic_train, df_atomic_test)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Data Processing completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_desc_train.shape)
print(df_desc_test.shape)
print(df_fp_train.shape)
print(df_fp_test.shape)
print(df_emb_train.shape)
print(df_emb_test.shape)
print(df_atomic_train.shape)
print(df_atomic_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_desc_train)
print(df_desc_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_fp_train)
print(df_fp_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_emb_train)
print(df_emb_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_atomic_train)
print(df_atomic_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')


/tmp/ipykernel_2667770/1337071642.py:2: DtypeWarning: Columns (1275,1277,1280,1285,1292,1298,1304,1354,1356,1359,1364,1371,1377,1383,1579,1580,1581,1583,1584,1590,1595,1596,1597) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desc_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Train_2d_3d_all_descriptors.csv')
/tmp/ipykernel_2667770/1337071642.py:10: DtypeWarning: Columns (1275,1277,1292,1298,1354,1356,1371,1377,1579,1580,1581,1583,1584,1595,1596,1597) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desc_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Test_2d_3d_all_descriptors.csv')


XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Data Loading completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
(5568, 271)
(1392, 271)
(5568, 1128)
(1392, 1128)
(5568, 758)
(1392, 758)
(5568, 12)
(1392, 12)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Data Processing completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
(5568, 271)
(1392, 271)
(5568, 1128)
(1392, 1128)
(5568, 758)
(1392, 758)
(5568, 12)
(1392, 12)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
        ID                                             SMILES  Permeability  \
4765     3  CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...     -7.000000   
4772     4  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...     -7.100000   
4766     6  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...     -7.300000   
476

In [11]:
models_weak = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500),
    DecisionTreeRegressor(random_state=101),

]

models_meta = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500)
]

dataframes = [(df_desc_train, df_desc_test), (df_fp_train, df_fp_test), (df_emb_train, df_emb_test), (df_atomic_train, df_atomic_test)]

meta_features_train = []
meta_features_test = []

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1: Training weak learners with 15-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Stage 1: Train weak learners with 15-fold cross-validation
for df_train, df_test in tqdm(dataframes, desc="Processing dataframe pairs"):
    X_weak = df_train.drop(columns=['ID', 'SMILES', target_column])
    y_weak = df_train[target_column]
    X_eval = df_test.drop(columns=['ID', 'SMILES', target_column])
    y_eval = df_test[target_column]

    kf = KFold(n_splits=15, shuffle=True, random_state=101)

    fold_meta_features_train = np.zeros((X_weak.shape[0], len(models_weak)))
    fold_meta_features_test = np.zeros((X_eval.shape[0], len(models_weak)))

    for i, model in tqdm(enumerate(models_weak), desc="Training models", total=len(models_weak)):
        fold_predictions = np.zeros(X_weak.shape[0])
        test_predictions_folds = []

        for train_index, val_index in kf.split(X_weak):
            X_train, X_val = X_weak.iloc[train_index], X_weak.iloc[val_index]
            y_train, y_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

            model.fit(X_train, y_train)

            fold_predictions[val_index] = np.clip(model.predict(X_val), -10, -3.9)
            test_predictions_fold = np.clip(model.predict(X_eval), -10, -3.9)
            test_predictions_folds.append(test_predictions_fold)

        fold_meta_features_train[:, i] = fold_predictions
        fold_meta_features_test[:, i] = np.mean(test_predictions_folds, axis=0)
        print(f'Model training done {i}')

    meta_features_train.append(fold_meta_features_train)
    meta_features_test.append(fold_meta_features_test)

meta_features_train = np.hstack(meta_features_train)
meta_features_test = np.hstack(meta_features_test)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print("Dimensions of meta_features_train:", meta_features_train.shape)
print("Dimensions of meta_features_test:", meta_features_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1 completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 2: Training meta-learners')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')


results = {}
predictions = []

kf = KFold(n_splits=15, shuffle=True, random_state=101)

for model in models_meta:
    model_name = model.__class__.__name__
    predictions_train = []
    actual_y_train = []
    test_predictions_folds = []

    for train_index, val_index in kf.split(meta_features_train):
        X_fold_train, X_fold_val = meta_features_train[train_index], meta_features_train[val_index]
        y_fold_train, y_fold_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

        model.fit(X_fold_train, y_fold_train)
        y_pred_fold = model.predict(X_fold_val)
        y_pred_fold = np.clip(y_pred_fold, -10, -3.9)

        predictions_train.extend(y_pred_fold)
        actual_y_train.extend(y_fold_val)

        test_predictions_fold = model.predict(meta_features_test)
        test_predictions_fold = np.clip(test_predictions_fold, -10, -3.9)
        test_predictions_folds.append(test_predictions_fold)

    predictions_test_mean = np.mean(test_predictions_folds, axis=0)
    predictions_test_std = np.std(test_predictions_folds, axis=0)

    mse_train = mean_squared_error(actual_y_train, predictions_train)
    mae_train = mean_absolute_error(actual_y_train, predictions_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(actual_y_train, predictions_train)
    pearson_train, _ = pearsonr(actual_y_train, predictions_train)
    spearman_train, _ = spearmanr(actual_y_train, predictions_train)

    mse_test = mean_squared_error(y_eval, predictions_test_mean)
    mae_test = mean_absolute_error(y_eval, predictions_test_mean)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_eval, predictions_test_mean)
    pearson_test, _ = pearsonr(y_eval, predictions_test_mean)
    spearman_test, _ = spearmanr(y_eval, predictions_test_mean)

    print(f'{model_name} Evaluation completed: Test R2 score: {r2_test:.4f}, Test RMSE: {rmse_test:.4f}')

    predictions.append({
        'Model': model_name,
        'Y Train pred': predictions_train,
        'Y Train actual': actual_y_train,
        'Y Test actual': y_eval,
        'Test Predictions folds': test_predictions_folds,
        'Test Predictions Mean': predictions_test_mean,
        'Test Predictions Std': predictions_test_std,
    })

    results[model_name] = {
        'Train MSE (15 fold CV)': mse_train,
        'Train MAE (15 fold CV)': mae_train,
        'Train RMSE (15 fold CV)': rmse_train,
        'Train R2 (15 fold CV)': r2_train,
        'Train PCC (15 fold CV)': pearson_train,
        'Train SCC (15 fold CV)': spearman_train,
        'Test MSE': mse_test,
        'Test MAE': mae_test,
        'Test RMSE': rmse_test,
        'Test R2': r2_test,
        'Test PCC': pearson_test,
        'Test SCC': spearman_test,
    }

results_df = pd.DataFrame(results).T
predictions_df = pd.DataFrame(predictions)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stacked Ensemble Training and Evaluation complete')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

results_df

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stage 1: Training weak learners with 15-fold CV
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


Training models:   0%|                                                                           | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008090 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56930
[LightGBM] [Info] Number of data points in the train set: 5196, number of used features: 262
[LightGBM] [Info] Start training from score -5.747798
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56947
[LightGBM] [Info] Number of data points in the train set: 5196, number of used features: 263
[LightGBM] [Info] Start training from score -5.741033
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007219 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56944
[LightGBM] [Info] Number of data points in the train set: 5196, number of used features: 263
[LightGBM] [Info] Star


Training models:  10%|██████▋                                                            | 1/10 [00:11<01:39, 11.11s/it]

Model training done 0


...

/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001674 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10139
[LightGBM] [Info] Number of data points in the train set: 5197, number of used features: 40
[LightGBM] [Info] Start training from score -5.734536


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor Evaluation completed: Test R2 score: 0.6957, Test RMSE: 0.4388
DecisionTreeRegressor Evaluation completed: Test R2 score: 0.6716, Test RMSE: 0.4558
RandomForestRegressor Evaluation completed: Test R2 score: 0.6930, Test RMSE: 0.4408
GradientBoostingRegressor Evaluation completed: Test R2 score: 0.6963, Test RMSE: 0.4384
AdaBoostRegressor Evaluation completed: Test R2 score: 0.6515, Test RMSE: 0.4696
XGBRegressor Evaluation completed: Test R2 score: 0.6944, Test RMSE: 0.4398
ExtraTreesRegressor Evaluation completed: Test R2 score: 0.6930, Test RMSE: 0.4408
LinearRegression Evaluation completed: Test R2 score: 0.6942, Test RMSE: 0.4399
KNeighborsRegressor Evaluation completed: Test R2 score: 0.6799, Test RMSE: 0.4501
SVR Evaluation completed: Test R2 score: 0.6939, Test RMSE: 0.4401
MLPRegressor Evaluation completed: Test R2 score: 0.6964, Test RMSE: 0.4383
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stacked Ensemble Training and 

,Train MSE (15 fold CV),Train MAE (15 fold CV),Train RMSE (15 fold CV),Train R2 (15 fold CV),Train PCC (15 fold CV),Train SCC (15 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.124803,0.256939,0.353275,0.799612,0.894216,0.877238,0.192570,0.307265,0.438829,0.695699,0.834195,0.819372
DecisionTreeRegressor,0.267856,0.376856,0.517548,0.569923,0.789705,0.766357,0.207799,0.321427,0.455849,0.671635,0.820121,0.803596
RandomForestRegressor,0.126177,0.258095,0.355214,0.797407,0.893001,0.875599,0.194302,0.308937,0.440797,0.692963,0.832664,0.815785
GradientBoostingRegressor,0.124109,0.255552,0.352291,0.800726,0.894840,0.877708,0.192199,0.306568,0.438405,0.696286,0.834582,0.819575
AdaBoostRegressor,0.173815,0.322050,0.416911,0.720918,0.861063,0.855866,0.220510,0.352285,0.469585,0.651549,0.816796,0.812278
XGBRegressor,0.140338,0.272763,0.374618,0.774668,0.880829,0.863864,0.193417,0.309127,0.439793,0.694360,0.833550,0.817321
ExtraTreesRegressor,0.124208,0.255444,0.352431,0.800568,0.894773,0.876582,0.194272,0.309753,0.440763,0.693010,0.832764,0.815438
LinearRegression,0.124418,0.254880,0.352730,0.800231,0.894559,0.877908,0.193488,0.307760,0.439873,0.694248,0.833375,0.818256
KNeighborsRegressor,0.147932,0.281681,0.384619,0.762476,0.873291,0.853887,0.202575,0.321204,0.450083,0.679890,0.825404,0.807349
SVR,0.132723,0.260151,0.364312,0.786895,0.887303,0.875771,0.193713,0.305304,0.440128,0.693893,0.834128,0.817935


In [12]:
results_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Results_15_fold_stacked_prediction_LVR_scaled.csv')
predictions_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Prediction_data_Meta_learners_15_fold_stacked_prediction_LVR_scaled.csv')

In [7]:
models_weak = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500),
    DecisionTreeRegressor(random_state=101)
]

models_meta = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500)
]
target_column = 'Permeability'
dataframes = [(df_desc_train, df_desc_test), (df_fp_train, df_fp_test), (df_emb_train, df_emb_test), (df_atomic_train, df_atomic_test)]

meta_features_train = []
meta_features_test = []

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1: Training weak learners with 5-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Stage 1: Train weak learners with 5-fold cross-validation
for df_train, df_test in tqdm(dataframes, desc="Processing dataframe pairs"):
    X_weak = df_train.drop(columns=['ID', 'SMILES', target_column])
    y_weak = df_train[target_column]
    X_eval = df_test.drop(columns=['ID', 'SMILES', target_column])
    y_eval = df_test[target_column]
    
    kf = KFold(n_splits=5, shuffle=True, random_state=101)

    fold_meta_features_train = np.zeros((X_weak.shape[0], len(models_weak)))
    fold_meta_features_test = np.zeros((X_eval.shape[0], len(models_weak)))

    for i, model in tqdm(enumerate(models_weak), desc="Training models", total=len(models_weak)):
        fold_predictions = np.zeros(X_weak.shape[0])
        test_predictions_folds = []

        for train_index, val_index in kf.split(X_weak):
            X_train, X_val = X_weak.iloc[train_index], X_weak.iloc[val_index]
            y_train, y_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

            model.fit(X_train, y_train)

            fold_predictions[val_index] = np.clip(model.predict(X_val), -10, -3.9)
            test_predictions_fold = np.clip(model.predict(X_eval), -10, -3.9)
            test_predictions_folds.append(test_predictions_fold)

        fold_meta_features_train[:, i] = fold_predictions
        fold_meta_features_test[:, i] = np.mean(test_predictions_folds, axis=0)
        print(f'Model training done {i}')

    meta_features_train.append(fold_meta_features_train)
    meta_features_test.append(fold_meta_features_test)

meta_features_train = np.hstack(meta_features_train)
meta_features_test = np.hstack(meta_features_test)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print("Dimensions of meta_features_train:", meta_features_train.shape)
print("Dimensions of meta_features_test:", meta_features_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1 completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 2: Training meta-learners with 5-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')


results = {}
predictions = []

kf = KFold(n_splits=5, shuffle=True, random_state=101)

for model in models_meta:
    model_name = model.__class__.__name__
    predictions_train = []
    actual_y_train = []
    test_predictions_folds = []

    for train_index, val_index in kf.split(meta_features_train):
        X_fold_train, X_fold_val = meta_features_train[train_index], meta_features_train[val_index]
        y_fold_train, y_fold_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

        model.fit(X_fold_train, y_fold_train)
        y_pred_fold = model.predict(X_fold_val)
        y_pred_fold = np.clip(y_pred_fold, -10, -3.9)

        predictions_train.extend(y_pred_fold)
        actual_y_train.extend(y_fold_val)

        test_predictions_fold = model.predict(meta_features_test)
        test_predictions_fold = np.clip(test_predictions_fold, -10, -3.9)
        test_predictions_folds.append(test_predictions_fold)

    predictions_test_mean = np.mean(test_predictions_folds, axis=0)
    predictions_test_std = np.std(test_predictions_folds, axis=0)

    mse_train = mean_squared_error(actual_y_train, predictions_train)
    mae_train = mean_absolute_error(actual_y_train, predictions_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(actual_y_train, predictions_train)
    pearson_train, _ = pearsonr(actual_y_train, predictions_train)
    spearman_train, _ = spearmanr(actual_y_train, predictions_train)

    
    mse_test = mean_squared_error(y_eval, predictions_test_mean)
    mae_test = mean_absolute_error(y_eval, predictions_test_mean)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_eval, predictions_test_mean)
    pearson_test, _ = pearsonr(y_eval, predictions_test_mean)
    spearman_test, _ = spearmanr(y_eval, predictions_test_mean)

    print(f'{model_name} Evaluation completed: Test R2 score: {r2_test:.4f}, Test RMSE: {rmse_test:.4f}')

    predictions.append({
        'Model': model_name,
        'Y Train pred': predictions_train,
        'Y Train actual': actual_y_train,
        'Y Test actual': y_eval,
        'Test Predictions folds': test_predictions_folds,
        'Test Predictions Mean': predictions_test_mean,
        'Test Predictions Std': predictions_test_std,
    })

    results[model_name] = {
        'Train MSE (5 fold CV)': mse_train,
        'Train MAE (5 fold CV)': mae_train,
        'Train RMSE (5 fold CV)': rmse_train,
        'Train R2 (5 fold CV)': r2_train,
        'Train PCC (5 fold CV)': pearson_train,
        'Train SCC (5 fold CV)': spearman_train,
        'Test MSE': mse_test,
        'Test MAE': mae_test,
        'Test RMSE': rmse_test,
        'Test R2': r2_test,
        'Test PCC': pearson_test,
        'Test SCC': spearman_test,
    }

results_df = pd.DataFrame(results).T
predictions_df = pd.DataFrame(predictions)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stacked Ensemble Training and Evaluation complete')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

results_df

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stage 1: Training weak learners with 5-fold CV
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


Training models:   0%|                                                                           | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008032 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56831
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 260
[LightGBM] [Info] Start training from score -5.747051
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007085 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56849
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 260
[LightGBM] [Info] Start training from score -5.747251
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007428 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56836
[LightGBM] [Info] Number of data points in the train set: 4454, number of used features: 261
[LightGBM] [Info] Star


Training models:  10%|██████▋                                                            | 1/10 [00:03<00:33,  3.67s/it]

Model training done 0


...

/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001522 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9961
[LightGBM] [Info] Number of data points in the train set: 4455, number of used features: 40
[LightGBM] [Info] Start training from score -5.726702


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor Evaluation completed: Test R2 score: 0.6926, Test RMSE: 0.4410
DecisionTreeRegressor Evaluation completed: Test R2 score: 0.6544, Test RMSE: 0.4676
RandomForestRegressor Evaluation completed: Test R2 score: 0.6888, Test RMSE: 0.4438
GradientBoostingRegressor Evaluation completed: Test R2 score: 0.6947, Test RMSE: 0.4395
AdaBoostRegressor Evaluation completed: Test R2 score: 0.6464, Test RMSE: 0.4730
XGBRegressor Evaluation completed: Test R2 score: 0.6804, Test RMSE: 0.4497
ExtraTreesRegressor Evaluation completed: Test R2 score: 0.6909, Test RMSE: 0.4423
LinearRegression Evaluation completed: Test R2 score: 0.6935, Test RMSE: 0.4404
KNeighborsRegressor Evaluation completed: Test R2 score: 0.6793, Test RMSE: 0.4505
SVR Evaluation completed: Test R2 score: 0.6933, Test RMSE: 0.4406
MLPRegressor Evaluation completed: Test R2 score: 0.6953, Test RMSE: 0.4391
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stacked Ensemble Training and 

,Train MSE (5 fold CV),Train MAE (5 fold CV),Train RMSE (5 fold CV),Train R2 (5 fold CV),Train PCC (5 fold CV),Train SCC (5 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.127861,0.259890,0.357577,0.794702,0.891473,0.876314,0.194517,0.309135,0.441041,0.692623,0.832415,0.818280
DecisionTreeRegressor,0.264635,0.373133,0.514427,0.575094,0.790962,0.773510,0.218677,0.331339,0.467629,0.654446,0.810925,0.793323
RandomForestRegressor,0.129418,0.261589,0.359747,0.792203,0.890144,0.873851,0.196953,0.309978,0.443793,0.688774,0.830284,0.814919
GradientBoostingRegressor,0.128346,0.260018,0.358254,0.793924,0.891025,0.875486,0.193185,0.308821,0.439529,0.694727,0.833655,0.818067
AdaBoostRegressor,0.173250,0.321010,0.416233,0.721825,0.859417,0.847435,0.223738,0.356501,0.473009,0.646448,0.811120,0.805100
XGBRegressor,0.146996,0.279729,0.383400,0.763980,0.875091,0.858649,0.202248,0.314479,0.449720,0.680406,0.825609,0.810231
ExtraTreesRegressor,0.128417,0.260089,0.358353,0.793810,0.891065,0.875282,0.195636,0.310127,0.442308,0.690854,0.831489,0.816471
LinearRegression,0.125450,0.256851,0.354189,0.798574,0.893633,0.876812,0.193970,0.309375,0.440420,0.693488,0.832908,0.817046
KNeighborsRegressor,0.153300,0.285699,0.391536,0.753856,0.868534,0.850334,0.202936,0.321619,0.450485,0.679318,0.824906,0.810051
SVR,0.135357,0.263448,0.367909,0.782667,0.884809,0.873899,0.194086,0.306082,0.440552,0.693303,0.833898,0.819824


In [8]:
results_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Results_5_fold_stacked_prediction_LVR_scaled.csv')
predictions_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Prediction_data_Meta_learners_5_fold_stacked_prediction_LVR_scaled.csv')

In [9]:
models_weak = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500),
    DecisionTreeRegressor(random_state=101)
]

models_meta = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500)
]
target_column = 'Permeability'
dataframes = [(df_desc_train, df_desc_test), (df_fp_train, df_fp_test), (df_emb_train, df_emb_test), (df_atomic_train, df_atomic_test)]

meta_features_train = []
meta_features_test = []

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1: Training weak learners with 10-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Stage 1: Train weak learners with 10-fold cross-validation
for df_train, df_test in tqdm(dataframes, desc="Processing dataframe pairs"):
    X_weak = df_train.drop(columns=['ID', 'SMILES', target_column])
    y_weak = df_train[target_column]
    X_eval = df_test.drop(columns=['ID', 'SMILES', target_column])
    y_eval = df_test[target_column]
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    fold_meta_features_train = np.zeros((X_weak.shape[0], len(models_weak)))
    fold_meta_features_test = np.zeros((X_eval.shape[0], len(models_weak)))

    for i, model in tqdm(enumerate(models_weak), desc="Training models", total=len(models_weak)):
        fold_predictions = np.zeros(X_weak.shape[0])
        test_predictions_folds = []

        for train_index, val_index in kf.split(X_weak):
            X_train, X_val = X_weak.iloc[train_index], X_weak.iloc[val_index]
            y_train, y_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

            model.fit(X_train, y_train)

            fold_predictions[val_index] = np.clip(model.predict(X_val), -10, -3.9)
            test_predictions_fold = np.clip(model.predict(X_eval), -10, -3.9)
            test_predictions_folds.append(test_predictions_fold)

        fold_meta_features_train[:, i] = fold_predictions
        fold_meta_features_test[:, i] = np.mean(test_predictions_folds, axis=0)
        print(f'Model training done {i}')

    meta_features_train.append(fold_meta_features_train)
    meta_features_test.append(fold_meta_features_test)

meta_features_train = np.hstack(meta_features_train)
meta_features_test = np.hstack(meta_features_test)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print("Dimensions of meta_features_train:", meta_features_train.shape)
print("Dimensions of meta_features_test:", meta_features_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1 completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 2: Training meta-learners with 10-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')


results = {}
predictions = []

kf = KFold(n_splits=10, shuffle=True, random_state=101)

for model in models_meta:
    model_name = model.__class__.__name__
    predictions_train = []
    actual_y_train = []
    test_predictions_folds = []

    for train_index, val_index in kf.split(meta_features_train):
        X_fold_train, X_fold_val = meta_features_train[train_index], meta_features_train[val_index]
        y_fold_train, y_fold_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

        model.fit(X_fold_train, y_fold_train)
        y_pred_fold = model.predict(X_fold_val)
        y_pred_fold = np.clip(y_pred_fold, -10, -3.9)

        predictions_train.extend(y_pred_fold)
        actual_y_train.extend(y_fold_val)

        test_predictions_fold = model.predict(meta_features_test)
        test_predictions_fold = np.clip(test_predictions_fold, -10, -3.9)
        test_predictions_folds.append(test_predictions_fold)

    predictions_test_mean = np.mean(test_predictions_folds, axis=0)
    predictions_test_std = np.std(test_predictions_folds, axis=0)

    mse_train = mean_squared_error(actual_y_train, predictions_train)
    mae_train = mean_absolute_error(actual_y_train, predictions_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(actual_y_train, predictions_train)
    pearson_train, _ = pearsonr(actual_y_train, predictions_train)
    spearman_train, _ = spearmanr(actual_y_train, predictions_train)

    
    mse_test = mean_squared_error(y_eval, predictions_test_mean)
    mae_test = mean_absolute_error(y_eval, predictions_test_mean)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_eval, predictions_test_mean)
    pearson_test, _ = pearsonr(y_eval, predictions_test_mean)
    spearman_test, _ = spearmanr(y_eval, predictions_test_mean)

    print(f'{model_name} Evaluation completed: Test R2 score: {r2_test:.4f}, Test RMSE: {rmse_test:.4f}')

    predictions.append({
        'Model': model_name,
        'Y Train pred': predictions_train,
        'Y Train actual': actual_y_train,
        'Y Test actual': y_eval,
        'Test Predictions folds': test_predictions_folds,
        'Test Predictions Mean': predictions_test_mean,
        'Test Predictions Std': predictions_test_std,
    })

    results[model_name] = {
        'Train MSE (10 fold CV)': mse_train,
        'Train MAE (10 fold CV)': mae_train,
        'Train RMSE (10 fold CV)': rmse_train,
        'Train R2 (10 fold CV)': r2_train,
        'Train PCC (10 fold CV)': pearson_train,
        'Train SCC (10 fold CV)': spearman_train,
        'Test MSE': mse_test,
        'Test MAE': mae_test,
        'Test RMSE': rmse_test,
        'Test R2': r2_test,
        'Test PCC': pearson_test,
        'Test SCC': spearman_test,
    }

results_df = pd.DataFrame(results).T
predictions_df = pd.DataFrame(predictions)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stacked Ensemble Training and Evaluation complete')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

results_df

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stage 1: Training weak learners with 10-fold CV
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


Training models:   0%|                                                                           | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007584 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56912
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 262
[LightGBM] [Info] Start training from score -5.747671
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007405 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56937
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 263
[LightGBM] [Info] Start training from score -5.741826
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56933
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 263
[LightGBM] [Info] Star


Training models:  10%|██████▋                                                            | 1/10 [00:07<01:09,  7.68s/it]

Model training done 0


...

/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001623 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10097
[LightGBM] [Info] Number of data points in the train set: 5012, number of used features: 40
[LightGBM] [Info] Start training from score -5.734177


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor Evaluation completed: Test R2 score: 0.6957, Test RMSE: 0.4388
DecisionTreeRegressor Evaluation completed: Test R2 score: 0.6612, Test RMSE: 0.4631
RandomForestRegressor Evaluation completed: Test R2 score: 0.6907, Test RMSE: 0.4425
GradientBoostingRegressor Evaluation completed: Test R2 score: 0.6965, Test RMSE: 0.4382
AdaBoostRegressor Evaluation completed: Test R2 score: 0.6657, Test RMSE: 0.4600
XGBRegressor Evaluation completed: Test R2 score: 0.6870, Test RMSE: 0.4451
ExtraTreesRegressor Evaluation completed: Test R2 score: 0.6928, Test RMSE: 0.4409
LinearRegression Evaluation completed: Test R2 score: 0.6941, Test RMSE: 0.4400
KNeighborsRegressor Evaluation completed: Test R2 score: 0.6817, Test RMSE: 0.4488
SVR Evaluation completed: Test R2 score: 0.6933, Test RMSE: 0.4406
MLPRegressor Evaluation completed: Test R2 score: 0.6903, Test RMSE: 0.4427
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stacked Ensemble Training and 

,Train MSE (10 fold CV),Train MAE (10 fold CV),Train RMSE (10 fold CV),Train R2 (10 fold CV),Train PCC (10 fold CV),Train SCC (10 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.124649,0.257625,0.353057,0.799860,0.894369,0.877403,0.192554,0.307291,0.438810,0.695725,0.834227,0.820651
DecisionTreeRegressor,0.265595,0.376011,0.515359,0.573552,0.789093,0.764616,0.214418,0.323827,0.463053,0.661175,0.815084,0.801343
RandomForestRegressor,0.126932,0.258329,0.356275,0.796194,0.892322,0.875365,0.195763,0.309286,0.442452,0.690653,0.831321,0.815909
GradientBoostingRegressor,0.124790,0.256373,0.353256,0.799633,0.894225,0.876869,0.192059,0.308014,0.438245,0.696507,0.834725,0.820596
AdaBoostRegressor,0.168182,0.315297,0.410101,0.729961,0.864573,0.852041,0.211558,0.345272,0.459955,0.665694,0.823699,0.812900
XGBRegressor,0.140547,0.273822,0.374896,0.774333,0.880612,0.862136,0.198070,0.311171,0.445051,0.687008,0.829228,0.815823
ExtraTreesRegressor,0.125389,0.256109,0.354103,0.798672,0.893708,0.876617,0.194397,0.309130,0.440905,0.692812,0.832603,0.816472
LinearRegression,0.124789,0.255568,0.353255,0.799634,0.894226,0.877550,0.193594,0.308208,0.439993,0.694081,0.833284,0.817977
KNeighborsRegressor,0.151814,0.285390,0.389633,0.756243,0.869643,0.850880,0.201411,0.321141,0.448789,0.681728,0.826085,0.804292
SVR,0.133759,0.261469,0.365730,0.785233,0.886403,0.875806,0.194117,0.305681,0.440587,0.693255,0.833838,0.819023


In [10]:
results_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Results_10_fold_stacked_prediction_LVR_scaled.csv')
predictions_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Prediction_data_Meta_learners_10_fold_stacked_prediction_LVR_scaled.csv')

In [5]:
models_weak = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500),
    DecisionTreeRegressor(random_state=101)
]

models_meta = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500)
]
target_column = 'Permeability'
dataframes = [(df_desc_train, df_desc_test), (df_fp_train, df_fp_test), (df_emb_train, df_emb_test), (df_atomic_train, df_atomic_test)]

meta_features_train = []
meta_features_test = []

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1: Training weak learners with 20-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Stage 1: Train weak learners with 20-fold cross-validation
for df_train, df_test in tqdm(dataframes, desc="Processing dataframe pairs"):
    X_weak = df_train.drop(columns=['ID', 'SMILES', target_column])
    y_weak = df_train[target_column]
    X_eval = df_test.drop(columns=['ID', 'SMILES', target_column])
    y_eval = df_test[target_column]
    kf = KFold(n_splits=20, shuffle=True, random_state=101)

    fold_meta_features_train = np.zeros((X_weak.shape[0], len(models_weak)))
    fold_meta_features_test = np.zeros((X_eval.shape[0], len(models_weak)))

    for i, model in tqdm(enumerate(models_weak), desc="Training models", total=len(models_weak)):
        fold_predictions = np.zeros(X_weak.shape[0])
        test_predictions_folds = []

        for train_index, val_index in kf.split(X_weak):
            X_train, X_val = X_weak.iloc[train_index], X_weak.iloc[val_index]
            y_train, y_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

            model.fit(X_train, y_train)

            fold_predictions[val_index] = np.clip(model.predict(X_val), -10, -3.9)
            test_predictions_fold = np.clip(model.predict(X_eval), -10, -3.9)
            test_predictions_folds.append(test_predictions_fold)

        fold_meta_features_train[:, i] = fold_predictions
        fold_meta_features_test[:, i] = np.mean(test_predictions_folds, axis=0)
        print(f'Model training done {i}')

    meta_features_train.append(fold_meta_features_train)
    meta_features_test.append(fold_meta_features_test)

meta_features_train = np.hstack(meta_features_train)
meta_features_test = np.hstack(meta_features_test)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print("Dimensions of meta_features_train:", meta_features_train.shape)
print("Dimensions of meta_features_test:", meta_features_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1 completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 2: Training meta-learners with 20-fold CV')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')


results = {}
predictions = []

kf = KFold(n_splits=20, shuffle=True, random_state=101)

for model in models_meta:
    model_name = model.__class__.__name__
    predictions_train = []
    actual_y_train = []
    test_predictions_folds = []

    for train_index, val_index in kf.split(meta_features_train):
        X_fold_train, X_fold_val = meta_features_train[train_index], meta_features_train[val_index]
        y_fold_train, y_fold_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

        model.fit(X_fold_train, y_fold_train)
        y_pred_fold = model.predict(X_fold_val)
        y_pred_fold = np.clip(y_pred_fold, -10, -3.9)

        predictions_train.extend(y_pred_fold)
        actual_y_train.extend(y_fold_val)

        test_predictions_fold = model.predict(meta_features_test)
        test_predictions_fold = np.clip(test_predictions_fold, -10, -3.9)
        test_predictions_folds.append(test_predictions_fold)

    predictions_test_mean = np.mean(test_predictions_folds, axis=0)
    predictions_test_std = np.std(test_predictions_folds, axis=0)

    mse_train = mean_squared_error(actual_y_train, predictions_train)
    mae_train = mean_absolute_error(actual_y_train, predictions_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(actual_y_train, predictions_train)
    pearson_train, _ = pearsonr(actual_y_train, predictions_train)
    spearman_train, _ = spearmanr(actual_y_train, predictions_train)

    
    mse_test = mean_squared_error(y_eval, predictions_test_mean)
    mae_test = mean_absolute_error(y_eval, predictions_test_mean)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_eval, predictions_test_mean)
    pearson_test, _ = pearsonr(y_eval, predictions_test_mean)
    spearman_test, _ = spearmanr(y_eval, predictions_test_mean)

    print(f'{model_name} Evaluation completed: Test R2 score: {r2_test:.4f}, Test RMSE: {rmse_test:.4f}')

    predictions.append({
        'Model': model_name,
        'Y Train pred': predictions_train,
        'Y Train actual': actual_y_train,
        'Y Test actual': y_eval,
        'Test Predictions folds': test_predictions_folds,
        'Test Predictions Mean': predictions_test_mean,
        'Test Predictions Std': predictions_test_std,

    })

    results[model_name] = {
        'Train MSE (20 fold CV)': mse_train,
        'Train MAE (20 fold CV)': mae_train,
        'Train RMSE (20 fold CV)': rmse_train,
        'Train R2 (20 fold CV)': r2_train,
        'Train PCC (20 fold CV)': pearson_train,
        'Train SCC (20 fold CV)': spearman_train,
        'Test MSE': mse_test,
        'Test MAE': mae_test,
        'Test RMSE': rmse_test,
        'Test R2': r2_test,
        'Test PCC': pearson_test,
        'Test SCC': spearman_test,
    }

results_df = pd.DataFrame(results).T
predictions_df = pd.DataFrame(predictions)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stacked Ensemble Training and Evaluation complete')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

results_df

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stage 1: Training weak learners with 20-fold CV
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


Training models:   0%|          | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.029310 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 56958
[LightGBM] [Info] Number of data points in the train set: 5289, number of used features: 263
[LightGBM] [Info] Start training from score -5.747281
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009334 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56944
[LightGBM] [Info] Number of data points in the train set: 5289, number of used features: 263
[LightGBM] [Info] Start training from score -5.743197
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007794 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56956
[LightGBM] [Info] Number of data points in the trai


Training models:  10%|█         | 1/10 [00:18<02:45, 18.44s/it]

Model training done 0


...

/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001568 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10164
[LightGBM] [Info] Number of data points in the train set: 5290, number of used features: 40
[LightGBM] [Info] Start training from score -5.736521


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


LGBMRegressor Evaluation completed: Test R2 score: 0.6972, Test RMSE: 0.4377
DecisionTreeRegressor Evaluation completed: Test R2 score: 0.6674, Test RMSE: 0.4588
RandomForestRegressor Evaluation completed: Test R2 score: 0.6926, Test RMSE: 0.4411
GradientBoostingRegressor Evaluation completed: Test R2 score: 0.6952, Test RMSE: 0.4392
AdaBoostRegressor Evaluation completed: Test R2 score: 0.6557, Test RMSE: 0.4668
XGBRegressor Evaluation completed: Test R2 score: 0.6931, Test RMSE: 0.4407
ExtraTreesRegressor Evaluation completed: Test R2 score: 0.6942, Test RMSE: 0.4399
LinearRegression Evaluation completed: Test R2 score: 0.6962, Test RMSE: 0.4385
KNeighborsRegressor Evaluation completed: Test R2 score: 0.6772, Test RMSE: 0.4520
SVR Evaluation completed: Test R2 score: 0.6925, Test RMSE: 0.4411
MLPRegressor Evaluation completed: Test R2 score: 0.6987, Test RMSE: 0.4367
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Stacked Ensemble Training and 

,Train MSE (20 fold CV),Train MAE (20 fold CV),Train RMSE (20 fold CV),Train R2 (20 fold CV),Train PCC (20 fold CV),Train SCC (20 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.123625,0.255786,0.351603,0.801504,0.895277,0.878916,0.191610,0.307490,0.437732,0.697217,0.835119,0.820597
DecisionTreeRegressor,0.255956,0.370629,0.505921,0.589030,0.798817,0.770858,0.210502,0.323504,0.458805,0.667363,0.818505,0.798929
RandomForestRegressor,0.125116,0.256122,0.353718,0.799109,0.893964,0.877420,0.194562,0.308341,0.441091,0.692552,0.832448,0.815556
GradientBoostingRegressor,0.124204,0.256301,0.352426,0.800574,0.894748,0.877182,0.192875,0.307301,0.439175,0.695218,0.833927,0.819573
AdaBoostRegressor,0.165833,0.311031,0.407225,0.733734,0.866184,0.858277,0.217909,0.347540,0.466807,0.655659,0.816905,0.811944
XGBRegressor,0.142237,0.274186,0.377143,0.771620,0.878973,0.862483,0.194220,0.307843,0.440704,0.693093,0.832948,0.818835
ExtraTreesRegressor,0.123368,0.254954,0.351238,0.801916,0.895523,0.877273,0.193505,0.308350,0.439892,0.694222,0.833482,0.816034
LinearRegression,0.123873,0.254337,0.351956,0.801106,0.895048,0.878397,0.192259,0.307641,0.438474,0.696190,0.834527,0.818674
KNeighborsRegressor,0.148924,0.280575,0.385906,0.760884,0.872412,0.852664,0.204262,0.317270,0.451954,0.677223,0.824058,0.809096
SVR,0.131766,0.259583,0.362996,0.788433,0.888171,0.877363,0.194604,0.305052,0.441139,0.692485,0.833372,0.818657


In [6]:
results_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Results_20_fold_stacked_prediction_LVR_scaled.csv')
predictions_df.to_csv('/home/users/akshay/PCPpred/PAMPA/Results/Stacked_architecture/Prediction_data_Meta_learners_20_fold_stacked_prediction_LVR_scaled.csv')

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import joblib


os.makedirs('/home/users/akshay/PCPpred/PAMPA/pampa_models/', exist_ok=True)

# 2D and 3D descriptors dataframes
df_desc_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Train_2d_3d_all_descriptors.csv')
df_train = df_desc_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'], axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features_desc = features(train, "Permeability")
joblib.dump(selected_features_desc, '/home/users/akshay/PCPpred/PAMPA/pampa_models/selected_features_descriptors.joblib')
df_desc_train = pd.concat([df_train[['ID','SMILES','Permeability']], df_train[selected_features_desc]], axis=1)
df_desc_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Test_2d_3d_all_descriptors.csv')
df_desc_test = df_desc_test.sort_values(by='ID')
df_desc_test = df_desc_test.dropna()
df_desc_test = df_desc_test[df_desc_train.columns]

# Fingerprints
df_fp_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/All_fingerprints_train.csv')
df_train = df_fp_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'], axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features_fp = features(train, "Permeability")
joblib.dump(selected_features_fp, '/home/users/akshay/PCPpred/PAMPA/pampa_models/selected_features_fingerprints.joblib')
df_fp_train = pd.concat([df_train[['ID','SMILES','Permeability']], df_train[selected_features_fp]], axis=1)
df_fp_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/All_fingerprints_test.csv')
df_fp_test = df_fp_test.sort_values(by='ID')
df_fp_test = df_fp_test.dropna()
df_fp_test = df_fp_test[df_fp_train.columns]

# Smiles Embeddings
df_emb_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Embeddings/Train_MoLFormer-XL-both-10pct_model_1_fine_tuned_embeddings.csv')
df_train = df_emb_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'], axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features_emb = features(train, "Permeability")
joblib.dump(selected_features_emb, '/home/users/akshay/PCPpred/PAMPA/pampa_models/selected_features_embeddings.joblib')
df_emb_train = pd.concat([df_train[['ID','SMILES','Permeability']], df_train[selected_features_emb]], axis=1)
df_emb_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Embeddings/Test_MoLFormer-XL-both-10pct_model_1_fine_tuned_embeddings.csv')
df_emb_test = df_emb_test.sort_values(by='ID')
df_emb_test = df_emb_test.dropna()
df_emb_test = df_emb_test[df_emb_train.columns]

# Atomic features
df_atomic_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Atomic/Train_all_atomic_desc.csv')
df_train = df_atomic_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'], axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features_atomic = features(train, "Permeability")
joblib.dump(selected_features_atomic, '/home/users/akshay/PCPpred/PAMPA/pampa_models/selected_features_atomic.joblib')
df_atomic_train = pd.concat([df_train[['ID','SMILES','Permeability']], df_train[selected_features_atomic]], axis=1)
df_atomic_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Atomic/Test_all_atomic_desc.csv')
df_atomic_test = df_atomic_test.sort_values(by='ID')
df_atomic_test = df_atomic_test.dropna()
df_atomic_test = df_atomic_test[df_atomic_train.columns]

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Data Loading completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Filter dataframes to have consistent IDs
df_fp_test = df_fp_test[df_fp_test['ID'].isin(df_desc_test['ID'])]
df_fp_train = df_fp_train[df_fp_train['ID'].isin(df_desc_train['ID'])]
df_emb_test = df_emb_test[df_emb_test['ID'].isin(df_desc_test['ID'])]
df_emb_train = df_emb_train[df_emb_train['ID'].isin(df_desc_train['ID'])]
df_atomic_test = df_atomic_test[df_atomic_test['ID'].isin(df_desc_test['ID'])]
df_atomic_train = df_atomic_train[df_atomic_train['ID'].isin(df_desc_train['ID'])]

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Data Processing completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_desc_train.shape)
print(df_desc_test.shape)
print(df_fp_train.shape)
print(df_fp_test.shape)
print(df_emb_train.shape)
print(df_emb_test.shape)
print(df_atomic_train.shape)
print(df_atomic_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_desc_train)
print(df_desc_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_fp_train)
print(df_fp_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_emb_train)
print(df_emb_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_atomic_train)
print(df_atomic_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

target_column = 'Permeability'

def scale_features(df_train, df_test, feature_type):
    scaler = StandardScaler()
    train_features = df_train.drop(columns=['ID', 'SMILES', target_column])
    test_features = df_test.drop(columns=['ID', 'SMILES', target_column])
    scaler.fit(train_features)
    train_scaled = pd.DataFrame(scaler.transform(train_features), columns=train_features.columns, index=df_train.index)
    test_scaled = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns, index=df_test.index)
    df_train_scaled = pd.concat([df_train[['ID', 'SMILES', target_column]], train_scaled], axis=1)
    df_test_scaled = pd.concat([df_test[['ID', 'SMILES', target_column]], test_scaled], axis=1)
    # Save the scaler
    joblib.dump(scaler, f'/home/users/akshay/PCPpred/PAMPA/pampa_models/scaler_{feature_type}.joblib')
    return df_train_scaled, df_test_scaled

df_desc_train, df_desc_test = scale_features(df_desc_train, df_desc_test, 'Descriptor')
df_fp_train, df_fp_test = scale_features(df_fp_train, df_fp_test, 'Fingerprints')
df_emb_train, df_emb_test = scale_features(df_emb_train, df_emb_test, 'Embeddings')
df_atomic_train, df_atomic_test = scale_features(df_atomic_train, df_atomic_test , 'Atomic')

models_weak = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500),
    DecisionTreeRegressor(random_state=101),
]

models_meta = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101)
]

dataframes = [(df_desc_train, df_desc_test), (df_fp_train, df_fp_test), (df_emb_train, df_emb_test), (df_atomic_train, df_atomic_test)]
data_names = ['descriptors', 'fingerprints', 'embeddings', 'atomic']

meta_features_train = []
meta_features_test = []

# Stage 1: Train weak learners with 10-fold cross-validation
for df_idx, (df_train, df_test) in enumerate(tqdm(dataframes, desc="Processing dataframe pairs")):
    X_weak = df_train.drop(columns=['ID', 'SMILES', target_column])
    X_eval = df_test.drop(columns=['ID', 'SMILES', target_column])
    y_weak = df_train[target_column]
    y_eval = df_test[target_column]

    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    fold_meta_features_train = np.zeros((X_weak.shape[0], len(models_weak)))
    fold_meta_features_test = np.zeros((X_eval.shape[0], len(models_weak)))

    for i, model in tqdm(enumerate(models_weak), desc="Training models", total=len(models_weak)):
        fold_predictions = np.zeros(X_weak.shape[0])
        test_predictions_folds = []

        for fold_idx, (train_index, val_index) in enumerate(kf.split(X_weak)):
            X_train, X_val = X_weak.iloc[train_index], X_weak.iloc[val_index]
            y_train, y_val = y_weak.iloc[train_index], y_weak.iloc[val_index]
            
            model.fit(X_train, y_train)
            
            model_name = model.__class__.__name__
            joblib.dump(model, f'/home/users/akshay/PCPpred/PAMPA/pampa_models/weak_{data_names[df_idx]}_{model_name}_fold_{fold_idx}.joblib')

            fold_predictions[val_index] = np.clip(model.predict(X_val), -10, -3.9)

            test_predictions_fold = np.clip(model.predict(X_eval), -10, -3.9)
            test_predictions_folds.append(test_predictions_fold)

        fold_meta_features_train[:, i] = fold_predictions
        fold_meta_features_test[:, i] = np.mean(test_predictions_folds, axis=0)

    meta_features_train.append(fold_meta_features_train)
    meta_features_test.append(fold_meta_features_test)
    
    joblib.dump(fold_meta_features_train, f'/home/users/akshay/PCPpred/PAMPA/pampa_models/meta_features_train_{data_names[df_idx]}.joblib')
    joblib.dump(fold_meta_features_test, f'/home/users/akshay/PCPpred/PAMPA/pampa_models/meta_features_test_{data_names[df_idx]}.joblib')

meta_features_train = np.hstack(meta_features_train)
meta_features_test = np.hstack(meta_features_test)

joblib.dump(meta_features_train, '/home/users/akshay/PCPpred/PAMPA/pampa_models/meta_features_train_combined.joblib')
joblib.dump(meta_features_test, '/home/users/akshay/PCPpred/PAMPA/pampa_models/meta_features_test_combined.joblib')

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print("Dimensions of meta_features_train:", meta_features_train.shape)
print("Dimensions of meta_features_test:", meta_features_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Stage 1 completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# Stage 2: Train the meta-learner using predictions from weak learners
kf = KFold(n_splits=10, shuffle=True, random_state=101)
results = {}
predictions = []
for model in models_meta:
    model_name = model.__class__.__name__
    predictions_train = []
    actual_y_train = []
    
    test_predictions_folds = []

    for fold_idx, (train_index, val_index) in enumerate(kf.split(meta_features_train)):
        X_fold_train, X_fold_val = meta_features_train[train_index], meta_features_train[val_index]
        y_fold_train, y_fold_val = y_weak.iloc[train_index], y_weak.iloc[val_index]
        
        model.fit(X_fold_train, y_fold_train)
        
        joblib.dump(model, f'/home/users/akshay/PCPpred/PAMPA/pampa_models/meta_{model_name}_fold_{fold_idx}.joblib')

        y_pred_fold = model.predict(X_fold_val)
        y_pred_fold = np.clip(y_pred_fold, -10, -3.9)
        predictions_train.extend(y_pred_fold)
        actual_y_train.extend(y_fold_val)

        test_predictions_fold = model.predict(meta_features_test)
        test_predictions_fold = np.clip(test_predictions_fold, -10, -3.9)
        test_predictions_folds.append(test_predictions_fold)

    # Metrics
    predictions_test_mean = np.mean(test_predictions_folds, axis=0)
    predictions_test_std = np.std(test_predictions_folds, axis=0)

    mse_train = mean_squared_error(actual_y_train, predictions_train)
    mae_train = mean_absolute_error(actual_y_train, predictions_train)
    rmse_train = np.sqrt(mse_train)
    r2_train = r2_score(actual_y_train, predictions_train)
    pearson_train, _ = pearsonr(actual_y_train, predictions_train)
    spearman_train, _ = spearmanr(actual_y_train, predictions_train)

    mse_test = mean_squared_error(y_eval, predictions_test_mean)
    mae_test = mean_absolute_error(y_eval, predictions_test_mean)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(y_eval, predictions_test_mean)
    pearson_test, _ = pearsonr(y_eval, predictions_test_mean)
    spearman_test, _ = spearmanr(y_eval, predictions_test_mean)
    

    predictions.append({
        'Model': model_name,
        'Y Train pred': predictions_train,
        'Y Test actual': y_eval,
        'Test prediction folds': test_predictions_folds,
        'Test Predictions Mean': predictions_test_mean,
        'Test Predictions Std': predictions_test_mean,
    })

    results[model_name] = {
        'Train MSE (10 fold CV)': mse_train,
        'Train MAE (10 fold CV)': mae_train,
        'Train RMSE (10 fold CV)': rmse_train,
        'Train R2 (10 fold CV)': r2_train,
        'Train PCC (10 fold CV)': pearson_train,
        'Train SCC (10 fold CV)': spearman_train,
        'Test MSE': mse_test,
        'Test MAE': mae_test,
        'Test RMSE': rmse_test,
        'Test R2': r2_test,
        'Test PCC': pearson_test,
        'Test SCC': spearman_test,
    }

results_df = pd.DataFrame(results).T

/tmp/ipykernel_2667770/3964313023.py:24: DtypeWarning: Columns (1275,1277,1280,1285,1292,1298,1304,1354,1356,1359,1364,1371,1377,1383,1579,1580,1581,1583,1584,1590,1595,1596,1597) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desc_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Train_2d_3d_all_descriptors.csv')
/tmp/ipykernel_2667770/3964313023.py:33: DtypeWarning: Columns (1275,1277,1292,1298,1354,1356,1371,1377,1579,1580,1581,1583,1584,1595,1596,1597) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desc_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Test_2d_3d_all_descriptors.csv')


XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Data Loading completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Data Processing completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
(5568, 271)
(1392, 271)
(5568, 1128)
(1392, 1128)
(5568, 758)
(1392, 758)
(5568, 12)
(1392, 12)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
        ID                                             SMILES  Permeability  \
4765     3  CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...     -7.000000   
4772     4  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...     -7.100000   
4766     6  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...     -7.300000   
4767     7  CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...     -7.300000   
4585    11  COc1ccc(

Training models:   0%|          | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008198 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56912
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 262
[LightGBM] [Info] Start training from score -5.747671
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007835 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56937
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 263
[LightGBM] [Info] Start training from score -5.741826
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007705 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56933
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 263
[LightGBM] [Info] Star


Training models:   0%|          | 0/10 [00:00<?, ?it/s]

...

/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001711 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10091
[LightGBM] [Info] Number of data points in the train set: 5012, number of used features: 40
[LightGBM] [Info] Start training from score -5.737254


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001674 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10097
[LightGBM] [Info] Number of data points in the train set: 5012, number of used features: 40
[LightGBM] [Info] Start training from score -5.734177


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
#Ablations study

In [1]:
import os
import joblib
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression  # LogisticRegression is not used for regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def remove_low_variance_columns(df, threshold=0.005):
    # df = df.drop(['ID','SMILES','Permeability'],axis=1)
    variances = df.var()
    
    low_variance_columns = variances[variances < threshold].index.tolist()
    
    df_cleaned = df.drop(columns=low_variance_columns)
    
    return df_cleaned, low_variance_columns

def features(df, target_column='Permeability', threshold=0.9):
    correlation_matrix = df.corr()
    
    features_to_drop = set()
    
    for feature in correlation_matrix.columns:
        if feature == target_column:
            continue 
        target_corr = correlation_matrix[target_column][feature]
        
        for other_feature in correlation_matrix.columns:
            if other_feature == feature or other_feature == target_column:
                continue
            
            if abs(correlation_matrix[feature][other_feature]) > threshold:
                other_target_corr = correlation_matrix[target_column][other_feature]

                if abs(other_target_corr) < abs(target_corr):
                    features_to_drop.add(other_feature)
                else:
                    features_to_drop.add(feature)
    selected_features = [col for col in df.columns if col not in features_to_drop and col != target_column]
    
    return selected_features


In [3]:
# 2D and 3D descriptors dataframes
df_desc_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Train_2d_3d_all_descriptors.csv')
df_train = df_desc_train.sort_values(by='ID')
df_train =df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_desc_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
df_desc_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Test_2d_3d_all_descriptors.csv')
df_desc_test = df_desc_test.sort_values(by='ID')
df_desc_test =df_desc_test.dropna()
df_desc_test =  df_desc_test[df_desc_train.columns]


# Fingerprints
df_fp_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Train/All_fingerprints_train.csv')
df_train = df_fp_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_fp_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
df_fp_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Fingerprints/Test/All_fingerprints_test.csv')
df_fp_test = df_fp_test.sort_values(by='ID')
df_fp_test = df_fp_test.dropna()
df_fp_test =  df_fp_test[df_fp_train.columns]


#Smiles Embeddings
df_emb_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Embeddings/Train_MoLFormer-XL-both-10pct_model_1_fine_tuned_embeddings.csv')
df_train = df_emb_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_emb_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
df_emb_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Embeddings/Test_MoLFormer-XL-both-10pct_model_1_fine_tuned_embeddings.csv')
df_emb_test = df_emb_test.sort_values(by='ID')
df_emb_test = df_emb_test.dropna()
df_emb_test =  df_emb_test[df_emb_train.columns]

#ATomic features
df_atomic_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Atomic/Train_all_atomic_desc.csv')
df_train = df_atomic_train.sort_values(by='ID')
df_train = df_train.dropna()
train = df_train.drop(['ID','SMILES'],axis=1)
train = train.select_dtypes(include=['number'])
train, _ = remove_low_variance_columns(train)
selected_features = features(train, "Permeability")
df_atomic_train = pd.concat( [df_train[['ID','SMILES','Permeability']],df_train[selected_features] ], axis=1)
# df_atomic_train =pd.concat( [df_train['SMILES'], df_train.select_dtypes(include=['number'])], axis=1)
df_atomic_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Atomic/Test_all_atomic_desc.csv')
df_atomic_test = df_atomic_test.sort_values(by='ID')
df_atomic_test = df_atomic_test.dropna()
df_atomic_test =  df_atomic_test[df_atomic_train.columns]


print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Data Loading completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
df_fp_test = df_fp_test[df_fp_test['ID'].isin(df_desc_test['ID'])]
df_fp_train = df_fp_train[df_fp_train['ID'].isin(df_desc_train['ID'])]

df_emb_test = df_emb_test[df_emb_test['ID'].isin(df_desc_test['ID'])]
df_emb_train = df_emb_train[df_emb_train['ID'].isin(df_desc_train['ID'])]

df_atomic_test = df_atomic_test[df_atomic_test['ID'].isin(df_desc_test['ID'])]
df_atomic_train = df_atomic_train[df_atomic_train['ID'].isin(df_desc_train['ID'])]

print(df_desc_train.shape)
print(df_desc_test.shape)
print(df_fp_train.shape)
print(df_fp_test.shape)
print(df_emb_train.shape)
print(df_emb_test.shape)
print(df_atomic_train.shape)
print(df_atomic_test.shape)

def scale_features(df_train, df_test):
    scaler = StandardScaler()
    train_features = df_train.drop(columns=['ID', 'SMILES', target_column])
    test_features = df_test.drop(columns=['ID', 'SMILES', target_column])
    scaler.fit(train_features)
    train_scaled = pd.DataFrame(scaler.transform(train_features), columns=train_features.columns, index=df_train.index)
    test_scaled = pd.DataFrame(scaler.transform(test_features), columns=test_features.columns, index=df_test.index)
    df_train_scaled = pd.concat([df_train[['ID', 'SMILES', target_column]], train_scaled], axis=1)
    df_test_scaled = pd.concat([df_test[['ID', 'SMILES', target_column]], test_scaled], axis=1)
    return df_train_scaled, df_test_scaled

target_column = 'Permeability'
df_desc_train, df_desc_test = scale_features(df_desc_train, df_desc_test)
df_fp_train, df_fp_test = scale_features(df_fp_train, df_fp_test)
df_emb_train, df_emb_test = scale_features(df_emb_train, df_emb_test)
df_atomic_train, df_atomic_test = scale_features(df_atomic_train, df_atomic_test)

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Data Processing completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_desc_train.shape)
print(df_desc_test.shape)
print(df_fp_train.shape)
print(df_fp_test.shape)
print(df_emb_train.shape)
print(df_emb_test.shape)
print(df_atomic_train.shape)
print(df_atomic_test.shape)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_desc_train)
print(df_desc_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_fp_train)
print(df_fp_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_emb_train)
print(df_emb_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print(df_atomic_train)
print(df_atomic_test)
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')


/tmp/ipykernel_524149/3482494709.py:2: DtypeWarning: Columns (1275,1277,1280,1285,1292,1298,1304,1354,1356,1359,1364,1371,1377,1383,1579,1580,1581,1583,1584,1590,1595,1596,1597) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desc_train = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Train_2d_3d_all_descriptors.csv')
/tmp/ipykernel_524149/3482494709.py:10: DtypeWarning: Columns (1275,1277,1292,1298,1354,1356,1371,1377,1579,1580,1581,1583,1584,1595,1596,1597) have mixed types. Specify dtype option on import or set low_memory=False.
  df_desc_test = pd.read_csv('/home/users/akshay/PCPpred/PAMPA/features/Descriptors/Test_2d_3d_all_descriptors.csv')


XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Data Loading completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
(5568, 271)
(1392, 271)
(5568, 1128)
(1392, 1128)
(5568, 758)
(1392, 758)
(5568, 12)
(1392, 12)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Data Processing completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
(5568, 271)
(1392, 271)
(5568, 1128)
(1392, 1128)
(5568, 758)
(1392, 758)
(5568, 12)
(1392, 12)
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
        ID                                             SMILES  Permeability  \
4765     3  CC(C)C[C@H]1NC(=O)[C@H](Cc2ccc(O)cc2)NC(=O)[C@...     -7.000000   
4772     4  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...     -7.100000   
4766     6  CC(C)C[C@@H]1NC(=O)[C@H](CC(C)C)NC(=O)[C@H](Cc...     -7.300000   
476

In [4]:
models_weak = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500),
    DecisionTreeRegressor(random_state=101),

]
models_meta = [
    lgb.LGBMRegressor(objective='regression', metric='rmse', boosting_type='gbdt', num_leaves=31, learning_rate=0.05, random_state=101),
    DecisionTreeRegressor(random_state=101),
    RandomForestRegressor(n_jobs=-1, random_state=101),
    GradientBoostingRegressor(random_state=101),
    AdaBoostRegressor(random_state=101),
    xgb.XGBRegressor(random_state=101, eval_metric='rmse'),
    ExtraTreesRegressor(n_jobs=-1, n_estimators=100, random_state=101),
    LinearRegression(),
    KNeighborsRegressor(),
    SVR(),
    MLPRegressor(random_state=101, max_iter=500)
]

dataframes = [(df_desc_train, df_desc_test), (df_fp_train, df_fp_test), (df_emb_train, df_emb_test), (df_atomic_train, df_atomic_test)]

In [5]:
ablation_results = {}

for ablation_idx in range(len(dataframes)):
    print(f"========== Ablation: Excluding feature at index {ablation_idx} ==========")
    feature_names = ['Descriptor', 'Fingerprints', 'Embeddings', 'Atomic']
    print(f"========== Ablation: Excluding feature :-- {feature_names[ablation_idx]} ==========")

    ablated_dataframes = [pair for i, pair in enumerate(dataframes) if i != ablation_idx]

    meta_features_train = []
    meta_features_test = []

    # Stage 1
    for df_train, df_test in tqdm(ablated_dataframes, desc="Processing ablated dataframes"):
        X_weak = df_train.drop(columns=['ID', 'SMILES', target_column])
        y_weak = df_train[target_column]
        X_eval = df_test.drop(columns=['ID', 'SMILES', target_column])
        y_eval = df_test[target_column]

        kf = KFold(n_splits=10, shuffle=True, random_state=101)

        fold_meta_features_train = np.zeros((X_weak.shape[0], len(models_weak)))
        fold_meta_features_test = np.zeros((X_eval.shape[0], len(models_weak)))

        for i, model in tqdm(enumerate(models_weak), desc="Training weak models", total=len(models_weak)):
            fold_predictions = np.zeros(X_weak.shape[0])
            test_predictions_folds = []

            for train_index, val_index in kf.split(X_weak):
                X_train, X_val = X_weak.iloc[train_index], X_weak.iloc[val_index]
                y_train, y_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

                model.fit(X_train, y_train)

                fold_predictions[val_index] = np.clip(model.predict(X_val), -10, -3.9)
                test_predictions_fold = np.clip(model.predict(X_eval), -10, -3.9)
                test_predictions_folds.append(test_predictions_fold)

            fold_meta_features_train[:, i] = fold_predictions
            fold_meta_features_test[:, i] = np.mean(test_predictions_folds, axis=0)
            print(f'Model training done {i}: {model.__class__.__name__}')

        meta_features_train.append(fold_meta_features_train)
        meta_features_test.append(fold_meta_features_test)
        print('Dataframe training completed')

    # Stack all meta-features
    meta_features_train = np.hstack(meta_features_train)
    meta_features_test = np.hstack(meta_features_test)

    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
    print('Stage 1 completed (Weak Learners)')
    print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

    # Stage 2
    results = {}
    kf = KFold(n_splits=10, shuffle=True, random_state=101)

    for model in models_meta:
        model_name = model.__class__.__name__
        predictions_train = []
        actual_y_train = []
        test_predictions_folds = []

        for train_index, val_index in kf.split(meta_features_train):
            X_fold_train, X_fold_val = meta_features_train[train_index], meta_features_train[val_index]
            y_fold_train, y_fold_val = y_weak.iloc[train_index], y_weak.iloc[val_index]

            model.fit(X_fold_train, y_fold_train)
            y_pred_fold = np.clip(model.predict(X_fold_val), -10, -3.9)

            predictions_train.extend(y_pred_fold)
            actual_y_train.extend(y_fold_val)

            test_predictions_fold = model.predict(meta_features_test)
            test_predictions_fold = np.clip(test_predictions_fold, -10, -3.9)
            test_predictions_folds.append(test_predictions_fold)

        predictions_test_mean = np.mean(test_predictions_folds, axis=0)
        predictions_test_std = np.std(test_predictions_folds, axis=0)

        mse_train = mean_squared_error(actual_y_train, predictions_train)
        mae_train = mean_absolute_error(actual_y_train, predictions_train)
        rmse_train = np.sqrt(mse_train)
        r2_train = r2_score(actual_y_train, predictions_train)
        pearson_train, _ = pearsonr(actual_y_train, predictions_train)
        spearman_train, _ = spearmanr(actual_y_train, predictions_train)

        mse_test = mean_squared_error(y_eval, predictions_test_mean)
        mae_test = mean_absolute_error(y_eval, predictions_test_mean)
        rmse_test = np.sqrt(mse_test)
        r2_test = r2_score(y_eval, predictions_test_mean)
        pearson_test, _ = pearsonr(y_eval, predictions_test_mean)
        spearman_test, _ = spearmanr(y_eval, predictions_test_mean)

        results[model_name] = {
            'Train MSE (10 fold CV)': mse_train,
            'Train MAE (10 fold CV)': mae_train,
            'Train RMSE (10 fold CV)': rmse_train,
            'Train R2 (10 fold CV)': r2_train,
            'Train PCC (10 fold CV)': pearson_train,
            'Train SCC (10 fold CV)': spearman_train,
            'Test MSE': mse_test,
            'Test MAE': mae_test,
            'Test RMSE': rmse_test,
            'Test R2': r2_test,
            'Test PCC': pearson_test,
            'Test SCC': spearman_test,
        }

    ablation_results[f"Ablation_{feature_names[ablation_idx]}"] = pd.DataFrame(results).T

print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
print('Ablation Study Completed')
print('XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')

# To view the results
ablation_results_df = {key: value for key, value in ablation_results.items()}


========== Ablation: Excluding feature at index 0 ==========
========== Ablation: Excluding feature :-- Descriptor ==========


Training weak models:   0%|          | 0/10 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039811 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4196
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 1106
[LightGBM] [Info] Start training from score -5.747671
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.040476 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4207
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 1106
[LightGBM] [Info] Start training from score -5.741826
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.061030 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4200
[LightGBM] [Info] Number of data points in the train set: 5011, number of used features: 1105
[LightGBM] [Info] Star


Training weak models:  10%|█         | 1/10 [00:11<01:41, 11.30s/it]

Model training done 0: LGBMRegressor


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7629
[LightGBM] [Info] Number of data points in the train set: 5012, number of used features: 30
[LightGBM] [Info] Start training from score -5.737254


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001309 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7631
[LightGBM] [Info] Number of data points in the train set: 5012, number of used features: 30
[LightGBM] [Info] Start training from score -5.734177


/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/users/akshay/anaconda3/envs/py312/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
Ablation Study Completed
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [6]:
ablation_results

{'Ablation_Descriptor':                            Train MSE (10 fold CV)  Train MAE (10 fold CV)  \
 LGBMRegressor                            0.124547                0.257339   
 DecisionTreeRegressor                    0.271159                0.378883   
 RandomForestRegressor                    0.127370                0.258899   
 GradientBoostingRegressor                0.124091                0.256223   
 AdaBoostRegressor                        0.173011                0.317938   
 XGBRegressor                             0.141366                0.274843   
 ExtraTreesRegressor                      0.127268                0.258200   
 LinearRegression                         0.124801                0.255590   
 KNeighborsRegressor                      0.151503                0.284705   
 SVR                                      0.134686                0.262531   
 MLPRegressor                             0.135549                0.271661   
 
                            Train RMSE 

In [7]:
import os
import pickle

ablation_result_dir = '/home/users/akshay/PCPpred/PAMPA/Results/Ablation/'
os.makedirs(ablation_result_dir, exist_ok=True)

pickle_path = os.path.join(ablation_result_dir, 'ablation_results.pkl')
with open(pickle_path, 'wb') as f:
    pickle.dump(ablation_results, f)


with open(pickle_path, 'rb') as f:
    ablation_results = pickle.load(f)


ablation_results

{'Ablation_Descriptor':                            Train MSE (10 fold CV)  Train MAE (10 fold CV)  \
 LGBMRegressor                            0.124547                0.257339   
 DecisionTreeRegressor                    0.271159                0.378883   
 RandomForestRegressor                    0.127370                0.258899   
 GradientBoostingRegressor                0.124091                0.256223   
 AdaBoostRegressor                        0.173011                0.317938   
 XGBRegressor                             0.141366                0.274843   
 ExtraTreesRegressor                      0.127268                0.258200   
 LinearRegression                         0.124801                0.255590   
 KNeighborsRegressor                      0.151503                0.284705   
 SVR                                      0.134686                0.262531   
 MLPRegressor                             0.135549                0.271661   
 
                            Train RMSE 

In [8]:
ablation_result_dir = '/home/users/akshay/PCPpred/PAMPA/Results/Ablation/'
os.makedirs(ablation_result_dir, exist_ok=True)

for ablation_label, df in ablation_results.items():
    print(f"Results for {ablation_label}: \n")
    safe_label = ablation_label.replace(" ", "_").replace("/", "_")
    file_path = os.path.join(ablation_result_dir, f"{safe_label}.csv")
    df.to_csv(file_path)

Results for Ablation_Descriptor: 

Results for Ablation_Fingerprints: 

Results for Ablation_Embeddings: 

Results for Ablation_Atomic: 



In [9]:
from IPython.display import display
for ablation_label, df in ablation_results.items():
    print(f"Results for {ablation_label}: \n")
    display(df)

Results for Ablation_Descriptor: 



,Train MSE (10 fold CV),Train MAE (10 fold CV),Train RMSE (10 fold CV),Train R2 (10 fold CV),Train PCC (10 fold CV),Train SCC (10 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.124547,0.257339,0.352913,0.800023,0.894462,0.877640,0.193359,0.309692,0.439726,0.694453,0.833456,0.820650
DecisionTreeRegressor,0.271159,0.378883,0.520729,0.564619,0.784426,0.759031,0.216569,0.327040,0.465369,0.657777,0.812709,0.799445
RandomForestRegressor,0.127370,0.258899,0.356889,0.795491,0.891936,0.875326,0.196710,0.311360,0.443520,0.689158,0.830406,0.816779
GradientBoostingRegressor,0.124091,0.256223,0.352266,0.800755,0.894851,0.876724,0.194348,0.310251,0.440849,0.692890,0.832579,0.819734
AdaBoostRegressor,0.173011,0.317938,0.415946,0.722208,0.859052,0.850397,0.220874,0.352171,0.469972,0.650974,0.814813,0.809462
XGBRegressor,0.141366,0.274843,0.375987,0.773019,0.880031,0.862635,0.197695,0.313840,0.444629,0.687601,0.829688,0.816433
ExtraTreesRegressor,0.127268,0.258200,0.356747,0.795654,0.892023,0.874573,0.196641,0.311836,0.443442,0.689267,0.830498,0.816118
LinearRegression,0.124801,0.255590,0.353271,0.799617,0.894215,0.877585,0.195866,0.310431,0.442568,0.690491,0.831149,0.816576
KNeighborsRegressor,0.151503,0.284705,0.389234,0.756742,0.869998,0.849301,0.199434,0.320926,0.446581,0.684853,0.827842,0.805840
SVR,0.134686,0.262531,0.366995,0.783745,0.885559,0.874330,0.195332,0.307070,0.441963,0.691336,0.832594,0.818707


Results for Ablation_Fingerprints: 



,Train MSE (10 fold CV),Train MAE (10 fold CV),Train RMSE (10 fold CV),Train R2 (10 fold CV),Train PCC (10 fold CV),Train SCC (10 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.125781,0.257481,0.354656,0.798042,0.893349,0.876897,0.191068,0.306364,0.437114,0.698072,0.835614,0.820365
DecisionTreeRegressor,0.252359,0.368848,0.502353,0.594805,0.799321,0.775558,0.208692,0.322135,0.456828,0.670224,0.820248,0.799778
RandomForestRegressor,0.127854,0.259588,0.357567,0.794714,0.891494,0.874756,0.194917,0.309333,0.441494,0.691991,0.832140,0.815763
GradientBoostingRegressor,0.125087,0.257019,0.353676,0.799157,0.893964,0.876321,0.192349,0.307660,0.438576,0.696049,0.834473,0.820331
AdaBoostRegressor,0.173866,0.319790,0.416972,0.720836,0.859506,0.851227,0.214561,0.347395,0.463208,0.660949,0.821440,0.812258
XGBRegressor,0.141224,0.276066,0.375798,0.773246,0.879863,0.859806,0.195996,0.309806,0.442714,0.690286,0.831231,0.815661
ExtraTreesRegressor,0.127932,0.259208,0.357676,0.794588,0.891426,0.874096,0.193109,0.308256,0.439442,0.694847,0.833804,0.817990
LinearRegression,0.125036,0.255569,0.353604,0.799239,0.894004,0.877131,0.192681,0.308454,0.438955,0.695524,0.834174,0.819747
KNeighborsRegressor,0.149706,0.283514,0.386919,0.759627,0.871711,0.852315,0.207293,0.323132,0.455295,0.672434,0.821086,0.803682
SVR,0.132610,0.259854,0.364156,0.787078,0.887462,0.876649,0.194822,0.306770,0.441386,0.692141,0.833169,0.818860


Results for Ablation_Embeddings: 



,Train MSE (10 fold CV),Train MAE (10 fold CV),Train RMSE (10 fold CV),Train R2 (10 fold CV),Train PCC (10 fold CV),Train SCC (10 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.197204,0.325799,0.444077,0.683363,0.826659,0.798828,0.207748,0.326902,0.455794,0.671715,0.819593,0.806872
DecisionTreeRegressor,0.400688,0.458130,0.632999,0.356643,0.680875,0.656305,0.249740,0.355437,0.499740,0.605359,0.780632,0.762524
RandomForestRegressor,0.193206,0.322189,0.439552,0.689782,0.830666,0.804122,0.208753,0.326254,0.456895,0.670127,0.818713,0.803257
GradientBoostingRegressor,0.198992,0.327857,0.446085,0.680493,0.824943,0.795793,0.203710,0.324691,0.451342,0.678096,0.823480,0.808707
AdaBoostRegressor,0.259560,0.402047,0.509470,0.583243,0.789216,0.761066,0.247968,0.388195,0.497964,0.608159,0.804846,0.791111
XGBRegressor,0.219899,0.344418,0.468934,0.646924,0.806869,0.778609,0.219158,0.334292,0.468143,0.653685,0.809387,0.795459
ExtraTreesRegressor,0.186845,0.315897,0.432256,0.699996,0.836786,0.811588,0.211241,0.328015,0.459610,0.666195,0.816328,0.802155
LinearRegression,0.196513,0.325000,0.443298,0.684472,0.827342,0.797443,0.198451,0.318054,0.445478,0.686406,0.828584,0.814382
KNeighborsRegressor,0.222483,0.343583,0.471681,0.642775,0.802678,0.773852,0.235183,0.350971,0.484956,0.628363,0.794114,0.776606
SVR,0.208088,0.328868,0.456167,0.665888,0.817145,0.792233,0.206904,0.320719,0.454867,0.673048,0.821811,0.811838


Results for Ablation_Atomic: 



,Train MSE (10 fold CV),Train MAE (10 fold CV),Train RMSE (10 fold CV),Train R2 (10 fold CV),Train PCC (10 fold CV),Train SCC (10 fold CV),Test MSE,Test MAE,Test RMSE,Test R2,Test PCC,Test SCC
LGBMRegressor,0.125275,0.257694,0.353942,0.798855,0.893805,0.877471,0.193031,0.308192,0.439353,0.694971,0.833787,0.820005
DecisionTreeRegressor,0.263743,0.374735,0.513559,0.576526,0.792243,0.768514,0.212271,0.322619,0.460729,0.664568,0.816842,0.803776
RandomForestRegressor,0.127749,0.259039,0.357421,0.794882,0.891594,0.875051,0.195506,0.309744,0.442161,0.691060,0.831602,0.816087
GradientBoostingRegressor,0.125566,0.257346,0.354353,0.798388,0.893527,0.876989,0.191852,0.308384,0.438009,0.696834,0.834908,0.819767
AdaBoostRegressor,0.178632,0.325035,0.422649,0.713183,0.857310,0.848489,0.220008,0.355082,0.469050,0.652342,0.817703,0.807913
XGBRegressor,0.141496,0.275541,0.376159,0.772810,0.879515,0.862526,0.197481,0.312857,0.444389,0.687939,0.829882,0.815774
ExtraTreesRegressor,0.126231,0.257413,0.355290,0.797320,0.892954,0.875876,0.194766,0.309874,0.441323,0.692229,0.832291,0.815771
LinearRegression,0.124419,0.255014,0.352731,0.800229,0.894557,0.877798,0.192792,0.307843,0.439081,0.695348,0.834029,0.817838
KNeighborsRegressor,0.151292,0.285708,0.388962,0.757082,0.870208,0.848718,0.201650,0.322872,0.449054,0.681351,0.825972,0.803488
SVR,0.131419,0.259541,0.362517,0.788990,0.888530,0.877179,0.192793,0.305884,0.439083,0.695346,0.834871,0.818940
